In [1]:
from omnia_ae.api import AEEnvironment, AEAPI

In [2]:
from typing import List, Literal, Optional
from azure.identity._internal.msal_credentials import MsalCredential

In [3]:
from typing import Literal, Optional, TypedDict, Union, Dict, Any
from azure.identity._internal.msal_credentials import MsalCredential
import requests
import logging

from importlib import metadata
from opentelemetry.instrumentation.requests import RequestsInstrumentor
import platform

In [4]:
from azure.identity import ClientSecretCredential
import os
credential = ClientSecretCredential(
    tenant_id=os.environ['AZURE_TENANT_ID'],
    client_id=os.environ['AZURE_CLIENT_ID_NON_PRODUCTION'],
    client_secret=os.environ['AZURE_CLIENT_SECRET_NON_PRODUCTION']
)

In [5]:
api = AEAPI(azure_credential=credential, environment=AEEnvironment.Dev())

In [6]:
api.get_sources()

{'items': [{'facility': 'AHA'},
  {'facility': 'ASGA'},
  {'facility': 'ASGB'},
  {'facility': 'GFA'},
  {'facility': 'GFB'},
  {'facility': 'GFC'},
  {'facility': 'GKR'},
  {'facility': 'HD'},
  {'facility': 'JSV'},
  {'facility': 'KRIS'},
  {'facility': 'MARA'},
  {'facility': 'MARB'},
  {'facility': 'MLA'},
  {'facility': 'NOR'},
  {'facility': 'OSF'},
  {'facility': 'TROA'},
  {'facility': 'VIS'}]}

In [8]:
api.get_events("AHA", "*", 10)

{'items': [{'timestamp': '2016-02-23T05:06:55.4670000Z',
   'sourceName': '70UA7010',
   'description': 'Common Gas Alarm Spar deck S900',
   'eventType': 'Simple',
   'eventCategory': '3439269159',
   'message': 'Abnormal condition',
   'severity': 302,
   'priority': None,
   'condition': None,
   'alarmState': None,
   'ackedState': None,
   'activeState': None,
   'suppressedOrShelved': None,
   'node': 'A',
   'processArea': 'FG',
   'actiontime': '2016-02-23T05:06:55.4670000Z'},
  {'timestamp': '2016-02-23T05:06:55.4670000Z',
   'sourceName': '71UI2601',
   'description': 'DV lower deck south released',
   'eventType': 'Simple',
   'eventCategory': '3439269159',
   'message': 'Normal state',
   'severity': 302,
   'priority': None,
   'condition': None,
   'alarmState': None,
   'ackedState': None,
   'activeState': None,
   'suppressedOrShelved': None,
   'node': 'A',
   'processArea': 'FG',
   'actiontime': '2016-02-23T05:06:55.4670000Z'},
  {'timestamp': '2016-02-23T05:06:55.4